Extract detailed information from detail_scrape

In [67]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import os
import datetime as dt
import sqlite3 as sq3

def sql_open():
    with sq3.connect('PropertyDetails_orininal.sql') as conn:
        return

In [60]:
q = 'select * from sqlite_master'
sql_open(q)

'select * from sqlite_master'

In [2]:
v = pd.read_csv('VkerDB.csv', usecols=['date']).max()[0]
print('The last date processed: {}'.format((v).strip()))
with

The last date processed: 2020-03-01


In [3]:
#file list filter by district
dist = 'V'

list_Hu =sorted([i for i in os.listdir() if 'adshu'+dist+'_' in i])
list_Com = sorted([i for i in os.listdir() if 'adscom'+dist+'_' in i])
list_Cen = sorted([i for i in os.listdir() if 'adscen'+dist+'_' in i])
print('List of files taken from .hu: \n', list(zip(range(len(list_Hu)),list_Hu)))
print('List of files taken from .com:\n', list(zip(range(len(list_Com)),list_Com)))
print('List of files taken from \'centrum\':\n', list(zip(range(len(list_Cen)) ,list_Cen)))

List of files taken from .hu: 
 [(0, 'adshuV_01032020.csv'), (1, 'adshuV_08032020.csv'), (2, 'adshuV_24022020.csv')]
List of files taken from .com:
 [(0, 'adscomV_01032020.csv'), (1, 'adscomV_08032020.csv'), (2, 'adscomV_24022020.csv')]
List of files taken from 'centrum':
 [(0, 'adscenV_01032020.csv'), (1, 'adscenV_08032020.csv'), (2, 'adscenV_24022020.csv')]


In [4]:
#data extraction from ingatlanok.hu website
HU = pd.DataFrame()
for i in list_Hu:   #file level list
    html = open(i, 'r').read()
    l = BeautifulSoup(html, 'lxml').find_all('div', attrs={'id': 'content-container'})
    for ii in l:   #ad level list
        value = {}
        base = [i for i in ii.find(attrs = 'estate-base-data').text.split('\n') if len(i) > 0]
        extra = [i for i in ii.find(attrs = {'id':'tab-content-3'}).text.split('\n') if len(i) >0]
    #mass feature extract   
        for n,v in zip(np.arange(0,len(base),2),np.arange(1,len(base),2)):
            value[base[n].replace(':','').strip().lower()] = base[v].strip()
            
        for n,v in zip(np.arange(0,len(extra),2),np.arange(1,len(extra),2)):
            value[extra[n].replace(':','').strip().lower()] = extra[v].strip()  
            
    #description
        d = ii.find(attrs = 'tab-content active-tab-content').text
        if len(d) > 0:
            value['description'] = d.strip()
        else: value['description'] = np.NAN
    #agency
        age = ii.find('div', attrs= 'seller-office')
        try:
            if len(age) > 0:
                value['agency'] = age.text
        except:
            value['agency'] = np.NAN
    #url
        value['url'] = 'https://ingatlanok.hu/' + value['azonosító']       
    #date     
        value['date'] = dt.datetime.date(dt.datetime.strptime(re.sub(r'\D', '', i), '%d%m%Y'))
    #source
        value['source'] = 'hu'
        df = pd.DataFrame(value, index = [0])
        HU = pd.concat((HU,df), axis = 0, ignore_index = True, sort = False)

In [5]:
HU.shape

(65, 31)

In [61]:
sql_open(HU.to_sql('hu', conn))

ProgrammingError: Cannot operate on a closed database.

In [63]:
q = 'select * from sqlite_master'
pd.read_sql(q,sql_open())

TypeError: sql_open() missing 1 required positional argument: 'q'

In [56]:
q= 'select * from cen'
pd.read_sql(q,conn)

ProgrammingError: Cannot operate on a closed database.

In [94]:
HU.isnull().sum()

ár                    0
méret                 0
azonosító             0
irodai azonosító     13
cím                   7
bútorozás             0
tulajdonviszony      50
építőanyag            7
egész szobák          0
fél szobák           50
komfort fokozat      39
ingatlan állapota     0
emelet               13
fűtés jellege        13
kilátás              18
építési év           48
erkély               39
garázs               59
pince                56
tároló               56
lift                 10
description           0
agency               10
url                   0
date                  0
source                0
építési mód          58
városrész            62
elhelyezkedés        62
kertkapcsolat        62
klíma                62
dtype: int64

In [6]:
#data extractio from ingatlan.com website
Com = pd.DataFrame()
for i in list_Com:
    html = open(i, 'r').read()
    l = BeautifulSoup(html, 'lxml').find_all('div', attrs = 'site__content')
    for ii in l:
        value = {}
        
        #id
        value['azonosító'] = ii.find(attrs = 'listing-id').text
        #url
        value['url'] = 'https://ingatlan.com/'+ value['azonosító']
        #agency
        try:
            value['irodai azonosító'] = ii.find(attrs = 'agent').find('a', attrs='partner-office-image partner-image')['href']
        except: value['irodai azonosító'] = np.NAN
        #address
        value['cím'] = ii.find(attrs = 'js-listing-title').text.strip()
        #sqm
        value['méret'] = re.findall('\d{2}',re.findall('\d{2}.m²',ii.find(attrs = 'parameter-area-size').text)[0])[0]
        #room
        value['egész szobák'] = re.findall('\d+,?\d?',ii.find(attrs = 'parameter-room').text)[0]
        #price
        value['ár'] = re.findall('\d{2},?\d?\d?',ii.find(attrs = 'parameter-price').text)[0].replace(',','.')
        #material
        value['építőanyag'] = ii.find(attrs = 'card details').h2.text.replace('Eladó ', '').replace(' lakás','')
        # description
        value['description'] = ii.find(attrs = 'long-description').text
        
        for t in range (2):
            tabls = ii.find(attrs = 'card details').find_all('table')[t].find_all('tr')
            for tr in range(len(tabls)):
                trs = tabls[tr].find_all('td')
                value[trs[0].text.strip().lower()] = trs[1].text.strip()
        

        #date     
        value['date'] = dt.datetime.date(dt.datetime.strptime(re.sub(r'\D', '', i), '%d%m%Y'))
        #source
        value['source'] = 'com'
        
        df = pd.DataFrame(value, index = [0])
        Com = pd.concat((Com,df), axis = 0, ignore_index = True, sort = False)


In [20]:
Com.to_sql('com',conn)

In [175]:
Com.isnull().sum()

azonosító               0
url                     0
agency                 57
cím                     0
méret                   0
egész szobák            0
ár                      0
építőanyag              0
description             0
ingatlan állapota       0
építés éve             40
komfort                 0
energiatanúsítvány      0
emelet                  0
épület szintjei         0
lift                    0
belmagasság             0
fűtés                   0
akadálymentesített      0
fürdő és wc             0
tájolás                 0
kilátás                 0
kertkapcsolatos         0
tetőtér                 0
parkolás                0
date                    0
source                  0
légkondicionáló        90
rezsiköltség           96
erkély                105
dtype: int64

In [7]:
#data extractio from ingatlan.com website
Cen = pd.DataFrame()
for i in list_Cen:
    html = open(i, 'r').read()
    l = BeautifulSoup(html, 'lxml').find_all(attrs={'id':'main'})
    for ii in l:
        value = {}
        #ID
        value['azonosító'] = ii.find(attrs = 'property-page')['data-reg-nr']
        #url
        value['url'] = 'https://www.oc.hu/ingatlanok/'+str(value['azonosító'])

        value['városrész'] = ii.find(attrs = 'estate-town atEstateListItem-Town').text
        head = ii.find(attrs = 'row estate-info-box hidden-xs')
        for n,m in zip(head.find_all('div',['left']),head.find_all('div',['right'])):
            value[n.text.replace('\n','').replace(':','').strip().lower()] = m.text.replace('\n','').strip()
            
        details = ii.findAll('li', ['col-sm-6'])
        for row in details:
            value[row.findAll('span')[0].text.replace(':','').lower()] = row.findAll('span')[1].text.strip()    

        #description
        value['decription'] = ii.find(attrs = 'description').text.replace('\n', '').strip()
        #date     
        value['date'] = dt.datetime.date(dt.datetime.strptime(re.sub(r'\D', '', i), '%d%m%Y'))
        #source
        value['source'] = 'cen'
        
        df = pd.DataFrame(value, index = [0])
        Cen = pd.concat((Cen,df), axis = 0, ignore_index = True, sort = False)
        

In [99]:
Cen.isnull().sum()

azonosító             0
url                   0
városrész             0
alapterület           0
szobák                0
regisztrációs szám    0
típus                 0
értékesítés típusa    0
méret (bruttó)        0
állapot               0
fűtés                 0
fürdőszobák száma     0
belmagasság           0
pince                 0
tájolás               0
környék               3
panoráma              3
kilátás               0
lépcsőház típusa      0
lift                  0
nappali-konyha        3
fürdőszoba-wc         3
galéria               3
decription            0
date                  0
source                0
nappali               3
konyha-étkező         3
dtype: int64

In [23]:
with sq3.connect('PropertyDetails.sql') as conn:
    Cen.to_sql('cen', conn)

/home/trismegistos/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [55]:
conn.close()